## Applied Capstone Project - Week 3 Peer Graded Assignment - Toronto Neighborhood

#### Instructions and guidelines

For this assignment, we were required to explore and cluster the neighborhoods in Toronto.
There were 4 steps or questions to be tackled as follows.

1. Creating a new Notebook for this assignment, I did this and dubbed the notebook "Neighborhoods-Toronto Canada"

2. Using the Notebook to build the code to scrape a Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

3. The pandas data frame should have the following:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository.

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

### Creating New Notebook and Importing Libraries

In [12]:
#Start by importing the libraries

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

!pip install bs4
!pip install lxml
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files
from pandas.io.json import json_normalize  # transform json files to pandas dataframes

!pip install folium
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


### Scraping data from Wikipedia page and storing it to a pandas Data Frame

In [2]:
#Sending the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#Parsing the data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

#Creating three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

#Appending the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))    # .rstrip('\n') avoids new lines in postalcode cell
        boroughList.append(cells[1].text.rstrip('\n'))       # .rstrip('\n') avoids new lines in borough cell
        neighborhoodList.append(cells[2].text.rstrip('\n'))  # .rstrip('\n') avoids new lines in neighborhood cell
        
#Creating a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Droping cells having Borough value of "Not Assigned"

In [3]:
#Drop cells having borough value of "Not assigned"
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Grouping the neighborhoods in the same Borough

In [4]:
#Grouping the neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### For the neighborhoods with "Not Assigned" values, make the value the same as the Borough

In [5]:
#If Neighborhood="Not assigned", make the value the same as that of Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Printing the number of rows

In [6]:
#Printing the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

### Loading the co-ordinates from the csv file provided

In [18]:
#Loading the coordinates from the csv file provided
coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the Geospatial csv data with the Toronto Data

In [19]:
#Renaming the column "PostalCode" so as to join with the Toronto data frame using the same column
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

#Merging the two table on the column "PostalCode" to make a comprehensive list including latitudes and longitudes
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Getting the Latitude and Longitude values of Toronto

In [21]:
#Using the geopy library to get the Latitude and Longitude values of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Plotting a map of Toronto with neighborhoods

In [22]:
#Plotting a map of Toronto using Latitude and Longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Adding markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Filtering for only Boroughs that contain the word 'Toronto'

In [23]:
#Filtering borough names that contain the word Toronto
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [24]:
#Creating a new dataframe with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Updating to plot a map of Toronto with the Boroughs that only have the word 'Toronto'

In [25]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Exploring the Neighborhoods using the Foursquare API

In [26]:
#Defining the Foursquare Credentials and Version
CLIENT_ID = 'XK5B5K5ZNFBQPJ5JL4HYUCT14FYIVW5W31WCW1YXGR0OWKXX' # your Foursquare ID
CLIENT_SECRET = 'RMSLYRWPTU2XIGO5L4REXILPM3TEZ4OTEK4MQ5YQPFBXYTCO' # your Foursquare Secret

#VERSION = 'YYYYMMDD' # Foursquare API version
VERSION = '20201121'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XK5B5K5ZNFBQPJ5JL4HYUCT14FYIVW5W31WCW1YXGR0OWKXX
CLIENT_SECRET:RMSLYRWPTU2XIGO5L4REXILPM3TEZ4OTEK4MQ5YQPFBXYTCO


#### Now, let's get the top 100 venues that are within a radius of 500 meters.

In [27]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [28]:
#Converting the venues list into a new dataframe
venues_df = pd.DataFrame(venues)

# defining the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1639, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### Checking how many venues were returned for each Postal Code

In [30]:
#Checking number of venues returned for each PostalCode in our dataframe
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",44,44,44,44,44,44
M4L,East Toronto,"India Bazaar, The Beaches West",19,19,19,19,19,19
M4M,East Toronto,Studio District,37,37,37,37,37,37
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,9,9,9,9,9,9
M4R,Central Toronto,"North Toronto West, Lawrence Park",18,18,18,18,18,18
M4S,Central Toronto,Davisville,34,34,34,34,34,34
M4T,Central Toronto,"Moore Park, Summerhill East",2,2,2,2,2,2


#### Checking for unique categories from the returned venues 

In [31]:
#Checking how many unique categories can be curated from all the returned venues highlighted above
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 240 uniques categories.


#### Analyzing Each Area

In [32]:
#One hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

#Add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

#Move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1639, 243)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 243)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000

#### Now let's create the new dataframe and display the top 10 venues for each PostalCode.

In [34]:
len(toronto_grouped[toronto_grouped["American Restaurant"] > 0])

10

In [35]:
toronto_american = toronto_grouped[["Neighborhoods","American Restaurant"]]
toronto_american.head()

,Neighborhoods,American Restaurant
0,The Beaches,0.000000
1,"The Danforth West, Riverdale",0.022727
2,"India Bazaar, The Beaches West",0.000000
3,Studio District,0.054054
4,Lawrence Park,0.000000


In [37]:
from sklearn.cluster import KMeans # importing library 
toclusters = 3  # 3 clusters selected

toronto_clustering = toronto_american.drop(["Neighborhoods"], 1)

#Run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(toronto_clustering)

#Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
      dtype=int32)

In [38]:
toronto_merged = toronto_american.copy()

#Add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_


#For merging two datasets make column name "neighborhood" in both dataset.
toronto_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True) 

#Lets check the above merged dataset
toronto_merged.head(5)

,Neighborhood,American Restaurant,Cluster Labels
0,The Beaches,0.000000,0
1,"The Danforth West, Riverdale",0.022727,1
2,"India Bazaar, The Beaches West",0.000000,0
3,Studio District,0.054054,2
4,Lawrence Park,0.000000,0


In [39]:
#merge the two dataframes on neighborhood column
toronto_merged = toronto_merged.join(venues_df.set_index("Neighborhood"), on="Neighborhood")

print(toronto_merged.shape)
toronto_merged.head()

(1639, 11)


,Neighborhood,American Restaurant,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,0.000000,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
0,The Beaches,0.000000,0,M4E,East Toronto,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
0,The Beaches,0.000000,0,M4E,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
0,The Beaches,0.000000,0,M4E,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
1,"The Danforth West, Riverdale",0.022727,1,M4K,East Toronto,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [40]:
# Lets sort the merged dataset on CLuster Labels basis
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged.head()

,Neighborhood,American Restaurant,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,0.0,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
26,"Kensington Market, Chinatown, Grange Park",0.0,0,M5T,Downtown Toronto,43.653206,-79.400049,Essence of Life Organics,43.654111,-79.400431,Organic Grocery
26,"Kensington Market, Chinatown, Grange Park",0.0,0,M5T,Downtown Toronto,43.653206,-79.400049,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,Mexican Restaurant
25,"University of Toronto, Harbord",0.0,0,M5S,Downtown Toronto,43.662696,-79.400049,Elchi Chai Shop,43.662695,-79.404652,Coffee Shop
25,"University of Toronto, Harbord",0.0,0,M5S,Downtown Toronto,43.662696,-79.400049,East of Brunswick,43.665609,-79.403324,Pub


In [41]:
#Plotting the map
map_clusters = folium.Map(location=[latitude, longitude],zoom_start=14)

#Set color scheme for the clusters


#Add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(toronto_merged['BoroughLatitude'], toronto_merged['BoroughLongitude'], toronto_merged['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's see the american restaurants clusterwise i.e. in each cluster

#### Cluster 0

In [42]:
#Cluster 0
toronto_merged.loc[(toronto_merged['Cluster Labels'] ==0) & (toronto_merged['VenueCategory'] == 'American Restaurant') ]

,Neighborhood,American Restaurant,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
28,Stn A PO Boxes,0.010417,0,M5W,Downtown Toronto,43.646435,-79.374846,Jump,43.648147,-79.378752,American Restaurant
12,Church and Wellesley,0.013333,0,M4Y,Downtown Toronto,43.665860,-79.383160,The Blake House,43.664468,-79.377471,American Restaurant


#### Cluster 1

In [43]:
#Cluster 1
toronto_merged.loc[(toronto_merged['Cluster Labels'] ==1) & (toronto_merged['VenueCategory'] == 'American Restaurant') ]

,Neighborhood,American Restaurant,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
29,"First Canadian Place, Underground city",0.030000,1,M5X,Downtown Toronto,43.648429,-79.382280,The Gabardine,43.650988,-79.381225,American Restaurant
29,"First Canadian Place, Underground city",0.030000,1,M5X,Downtown Toronto,43.648429,-79.382280,Richmond Station,43.651569,-79.379266,American Restaurant
29,"First Canadian Place, Underground city",0.030000,1,M5X,Downtown Toronto,43.648429,-79.382280,Cactus Club Cafe,43.649552,-79.381671,American Restaurant
1,"The Danforth West, Riverdale",0.022727,1,M4K,East Toronto,43.679557,-79.352188,Factory Girl,43.676693,-79.356299,American Restaurant
18,"Richmond, Adelaide, King",0.020000,1,M5H,Downtown Toronto,43.650571,-79.384568,Cactus Club Cafe,43.649552,-79.381671,American Restaurant
18,"Richmond, Adelaide, King",0.020000,1,M5H,Downtown Toronto,43.650571,-79.384568,Richmond Station,43.651569,-79.379266,American Restaurant
21,"Commerce Court, Victoria Hotel",0.030000,1,M5L,Downtown Toronto,43.648198,-79.379817,Cactus Club Cafe,43.649552,-79.381671,American Restaurant
20,"Toronto Dominion Centre, Design Exchange",0.030000,1,M5K,Downtown Toronto,43.647177,-79.381576,Jump,43.648147,-79.378752,American Restaurant
20,"Toronto Dominion Centre, Design Exchange",0.030000,1,M5K,Downtown Toronto,43.647177,-79.381576,The Gabardine,43.650988,-79.381225,American Restaurant
21,"Commerce Court, Victoria Hotel",0.030000,1,M5L,Downtown Toronto,43.648198,-79.379817,Richmond Station,43.651569,-79.379266,American Restaurant


#### Cluster 2

In [44]:
#Cluster 2
toronto_merged.loc[(toronto_merged['Cluster Labels'] ==2) & (toronto_merged['VenueCategory'] == 'American Restaurant') ]

,Neighborhood,American Restaurant,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
9,"Summerhill West, Rathnelly, South Hill, Forest...",0.071429,2,M4V,Central Toronto,43.686412,-79.400049,Union Social Eatery,43.687895,-79.394916,American Restaurant
3,Studio District,0.054054,2,M4M,East Toronto,43.659526,-79.340923,braised,43.660452,-79.343346,American Restaurant
3,Studio District,0.054054,2,M4M,East Toronto,43.659526,-79.340923,Brooklyn Tavern,43.661937,-79.335938,American Restaurant


### Observations

#### There are many American Restaurants in Cluster 1, mostly in Downtown Toronto
##### Assuming that Downtown Toronto is suitable for American Restaurants, then we could certainly look into Cluster 0 where there are few American Restaurants compared to the other two clusters
##### Looking at East Toronto and Central Toronto i.e. Cluster 2 we also see that American Restaurants could stand a chance there as well. In the neighborhood of Summerhill West, South Hill and the likes.